In [3]:
import httpx
from uuid import UUID
from typing import Optional
import os
from datetime import datetime, timezone

In [4]:
# Environment variable or fallback to localhost
BACKEND_URL = os.getenv("BACKEND_URL", "http://localhost:8000")
SESSION_ENDPOINT = f"{BACKEND_URL}/sessions"
client = httpx.Client(base_url=SESSION_ENDPOINT)

In [5]:
client = httpx.Client(base_url=SESSION_ENDPOINT)

In [6]:
client.get("runtime-env").json()

{'env': 'local',
 'file_exists': True,
 'dir_list': ['src',
  '.env',
  'vectordb',
  'LICENSE.txt',
  'README.md',
  'pyproject.toml'],
 'file_path': '.env'}

In [ ]:
client.get("/").json()

[{'id': 'da06a6dd-63df-410e-933c-e3c0e6c0ba5f',
  'label': 'Demo Contract Session',
  'created_at': '2025-06-07T21:50:07.117765',
  'updated_at': '2025-06-08T09:13:38.097353'}]

In [ ]:
def create_chat_session(label: str) -> Optional[UUID]:
    """Create a new chat session with the given label."""
    try:
        response = client.post("/", params={"label": label})
        response.raise_for_status()
        data = response.json()
        print(f"✅ Created session: {data['label']} ({data['id']})")
        return UUID(data["id"])
    except httpx.HTTPError as e:
        print(f"❌ Failed to create session: {e}")
        return None

In [8]:
# session_id = create_chat_session("Demo Contract Session")
# print(f"Session UUID: {session_id}")
session_id = "da06a6dd-63df-410e-933c-e3c0e6c0ba5f"

In [5]:
def send_message(session_id: UUID, role: str, content: str):
    data = {
        "role": role,
        "content": content,
    }

    response = client.post(f"/{session_id}/message", params=data)
    response.raise_for_status()
    return response.json()

In [28]:
response = send_message(session_id, role="user", content="Hello from client!")
print(response)

{'label': 'Demo Contract Session', 'updated_at': '2025-06-08T09:22:25.948600', 'messages': [{'role': 'user', 'content': 'Hello from client!', 'timestamp': '2025-06-08T08:46:42.363283+00:00'}, {'role': 'assistant', 'content': 'Hello from assistant!', 'timestamp': '2025-06-08T09:02:50.700660+00:00'}, {'role': 'user', 'content': 'Hello from client!', 'timestamp': '2025-06-08T09:06:47.523262+00:00'}, {'role': 'assistant', 'content': 'Hello from ASSISTANT!', 'timestamp': '2025-06-08T09:13:38.097114+00:00'}, {'role': 'user', 'content': 'Hello from client!', 'timestamp': '2025-06-08T09:22:25.948429+00:00'}], 'created_at': '2025-06-07T21:50:07.117765', 'id': 'da06a6dd-63df-410e-933c-e3c0e6c0ba5f'}


In [6]:
from typing import Dict

def get_chat_session(session_id: UUID) -> Optional[Dict]:
    """Fetch a chat session by its UUID using global HTTPX client."""
    try:
        response = client.get(f"/{session_id}")
        response.raise_for_status()
        data = response.json()
        print(f"✅ Fetched session '{data['label']}' with {len(data['messages'])} messages.")
        return data
    except httpx.HTTPStatusError as e:
        if e.response.status_code == 404:
            print("⚠️ Session not found.")
        else:
            print(f"❌ HTTP error occurred: {e}")
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
    return None


In [17]:
get_chat_session(session_id)

✅ Fetched session '5th Updated Demo Contract Session' with 7 messages.


{'messages': [{'role': 'user',
   'content': 'Hello from client!',
   'timestamp': '2025-06-08T08:46:42.363283+00:00'},
  {'role': 'assistant',
   'content': 'Hello from assistant!',
   'timestamp': '2025-06-08T09:02:50.700660+00:00'},
  {'role': 'user',
   'content': 'Hello from client!',
   'timestamp': '2025-06-08T09:06:47.523262+00:00'},
  {'role': 'assistant',
   'content': 'Hello from ASSISTANT!',
   'timestamp': '2025-06-08T09:13:38.097114+00:00'},
  {'role': 'user',
   'content': 'Hello from client!',
   'timestamp': '2025-06-08T09:22:25.948429+00:00'},
  {'role': 'user',
   'content': "What are the top 10 products for the category 'Electronics'?",
   'timestamp': '2025-06-08T10:38:02.898178+00:00'},
  {'role': 'assistant',
   'content': "Hi, good day! I received your question: What are the top 10 products for the category 'Electronics'?",
   'timestamp': '2025-06-08T10:38:02.898664+00:00'}],
 'label': '5th Updated Demo Contract Session',
 'updated_at': '2025-06-08T10:38:02.898

In [10]:
def rename_chat_session(session_id: UUID, new_label: str) -> dict:
    try:
        response = client.patch(f"/{session_id}", params={"label": new_label})
        response.raise_for_status()
        updated_session = response.json()
        print(f"✅ Renamed session to: {updated_session['label']}")
        return updated_session
    except Exception as e:
        print(f"❌ Failed to rename session: {e}")
        return {}


In [11]:
rename_chat_session(session_id, "4th Updated Demo Contract Session")

✅ Renamed session to: 4th Updated Demo Contract Session


{'label': '4th Updated Demo Contract Session',
 'updated_at': '2025-06-08T10:32:53.575364',
 'messages': [{'role': 'user',
   'content': 'Hello from client!',
   'timestamp': '2025-06-08T08:46:42.363283+00:00'},
  {'role': 'assistant',
   'content': 'Hello from assistant!',
   'timestamp': '2025-06-08T09:02:50.700660+00:00'},
  {'role': 'user',
   'content': 'Hello from client!',
   'timestamp': '2025-06-08T09:06:47.523262+00:00'},
  {'role': 'assistant',
   'content': 'Hello from ASSISTANT!',
   'timestamp': '2025-06-08T09:13:38.097114+00:00'},
  {'role': 'user',
   'content': 'Hello from client!',
   'timestamp': '2025-06-08T09:22:25.948429+00:00'}],
 'created_at': '2025-06-07T21:50:07.117765',
 'id': 'da06a6dd-63df-410e-933c-e3c0e6c0ba5f'}

In [12]:
def rename_chat_session(session_id: UUID, new_label: str) -> dict:
    """
    Rename a chat session by its UUID.
    
    Args:
        session_id (UUID): The ID of the session to rename.
        new_label (str): The new label for the session.

    Returns:
        dict: Updated session object (or error info).
    """
    try:
        response = client.patch(f"/{session_id}", params={"label": new_label})
        response.raise_for_status()
        updated_session = response.json()
        print(f"✅ Renamed session to: {updated_session['label']}")
        return updated_session
    except Exception as e:
        print(f"❌ Failed to rename session: {e}")
        return {}


In [13]:
rename_chat_session(session_id, "5th Updated Demo Contract Session")

✅ Renamed session to: 5th Updated Demo Contract Session


{'label': '5th Updated Demo Contract Session',
 'updated_at': '2025-06-08T10:33:32.088979',
 'messages': [{'role': 'user',
   'content': 'Hello from client!',
   'timestamp': '2025-06-08T08:46:42.363283+00:00'},
  {'role': 'assistant',
   'content': 'Hello from assistant!',
   'timestamp': '2025-06-08T09:02:50.700660+00:00'},
  {'role': 'user',
   'content': 'Hello from client!',
   'timestamp': '2025-06-08T09:06:47.523262+00:00'},
  {'role': 'assistant',
   'content': 'Hello from ASSISTANT!',
   'timestamp': '2025-06-08T09:13:38.097114+00:00'},
  {'role': 'user',
   'content': 'Hello from client!',
   'timestamp': '2025-06-08T09:22:25.948429+00:00'}],
 'created_at': '2025-06-07T21:50:07.117765',
 'id': 'da06a6dd-63df-410e-933c-e3c0e6c0ba5f'}

In [14]:
def send_chat_message(session_id: str, question: str) -> dict:
    """
    Send a message to the chat endpoint and receive a simulated assistant response.

    Args:
        session_id (str): The UUID of the chat session.
        question (str): The user's question to send.

    Returns:
        dict: Parsed response from the assistant including sources.
    """
    payload = {
        "session_id": session_id,
        "query": question
    }

    try:
        response = client.post("/chat", json=payload)
        response.raise_for_status()
        result = response.json()
        print(f"✅ Assistant response: {result['response']}")
        return result
    except httpx.HTTPError as e:
        print(f"❌ Error during chat request: {e}")
        return {}


In [15]:
response = send_chat_message(session_id, "What are the top 10 products for the category 'Electronics'?")

✅ Assistant response: Hi, good day! I received your question: What are the top 10 products for the category 'Electronics'?


In [16]:
response

{'response': "Hi, good day! I received your question: What are the top 10 products for the category 'Electronics'?",
 'success': True,
 'sources': [{'document_title': 'Dummy Source',
   'section_text': 'Dummy Section'}]}

In [18]:
def delete_chat_session(session_id: str) -> bool:
    try:
        response = client.delete(f"/{session_id}")
        response.raise_for_status()
        result = response.json()
        if result.get("success"):
            print(f"✅ Session {session_id} deleted successfully.")
            return True
        else:
            print(f"⚠️ Deletion failed for session {session_id}.")
            return False
    except httpx.HTTPStatusError as e:
        print(f"❌ HTTP error during deletion: {e.response.status_code} - {e.response.text}")
        return False
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
        return False


In [20]:
delete_chat_session(session_id)

❌ HTTP error during deletion: 404 - {"detail":"Session not found"}


False

In [1]:
def test_vector_db_search(question: str):
    endpoint = "/search"
    payload = {"question": question}

    try:
        response = client.post(endpoint, json=payload)
        response.raise_for_status()

        documents = response.json()
        print("Retrieved documents:")
        for doc in documents:
            print(f"- {doc['file_name']} (distance: {doc.get('_distance', doc.get('distance'))})")
        return documents

    except httpx.HTTPStatusError as e:
        print(f"HTTP error: {e.response.status_code} - {e.response.text}")
    except Exception as e:
        print(f"Request failed: {e}")

In [8]:
response = test_vector_db_search("What do you know about Supplier: Alpha Suppliers Inc. and what it supplies")

Retrieved documents:
- CW0302.pdf (distance: 0.9993271231651306)
- CW0303.pdf (distance: 1.025444507598877)
- CW0310.pdf (distance: 1.2309598922729492)
- CW0319.pdf (distance: 1.232763648033142)
- CW0315.pdf (distance: 1.2375408411026)


# Debugging

In [11]:
from procureme.dbmodels.session import ChatMessage, ChatRole, ChatSession
from pydantic_settings import BaseSettings
from pathlib import Path
from sqlmodel import Session, create_engine


In [12]:
DEFAULT_ENV_FILE = Path().absolute().parent.joinpath(".env")
DEFAULT_ENV_FILE.exists()
import os

In [13]:
class Settings(BaseSettings):
    OPENAI_API_KEY: str
    POSTGRES_USER: str
    POSTGRES_PASSWORD: str
    POSTGRES_HOST: str
    POSTGRES_PORT: str
    POSTGRES_DB: str


    @property
    def pg_database_url(self):
        host_system = os.getenv("HOST_SYSTEM")
        if host_system == "container":
            self.POSTGRES_HOST = "postgres"
            return (
                f"postgresql+psycopg2://{self.POSTGRES_USER}:"
                f"{self.POSTGRES_PASSWORD}@{self.POSTGRES_HOST}:"
                f"{self.POSTGRES_PORT}/{self.POSTGRES_DB}"
            )
        
        return (
            f"postgresql+psycopg2://{self.POSTGRES_USER}:"
            f"{self.POSTGRES_PASSWORD}@{self.POSTGRES_HOST}:"
            f"{self.POSTGRES_PORT}/{self.POSTGRES_DB}"
        )

    class Config:
        env_file = DEFAULT_ENV_FILE if os.getenv("RUNTIME_ENV", "local") == "local" else None
        extra = "ignore"


In [14]:
setting = Settings()

In [16]:
def get_session():
    settings = Settings()
    engine = create_engine(settings.pg_database_url, echo=True)
    return engine

In [ ]:
def append_message(session_id: UUID, role: ChatRole, content: str, db: Session = Depends(get_session)):
    chat = db.get(ChatSession, session_id)
    if not chat:
        raise HTTPException(status_code=404, detail="Session not found")
    logger.info(f"Received message: {chat}")
    message = ChatMessage(role=role, content=content, timestamp=datetime.now(tz=timezone.utc))
    logger.info(f"Received message: {message}")
    msg_json = message.model_dump()
    msg_json["timestamp"] = message.timestamp.isoformat()
    logger.info(f"Received message: {msg_json}")
    chat.messages.append(message.model_dump())
    chat.updated_at = datetime.now(tz=timezone.utc)
    logger.info(f"Received message: {chat}")
    db.add(chat)
    db.commit()
    db.refresh(chat)
    return chat

In [17]:
with Session(get_session()) as db:
    db_chat = db.get(ChatSession, UUID("da06a6dd-63df-410e-933c-e3c0e6c0ba5f"))
    message = ChatMessage(role=ChatRole.ASSISTANT, content="Hello from ASSISTANT!", timestamp=datetime.now(tz=timezone.utc).isoformat())
    db_chat.messages.append(message.model_dump())
    db_chat.updated_at = datetime.now(tz=timezone.utc)
    db_session = db_chat.model_dump(exclude_unset=True)
    db_chat.sqlmodel_update(db_session)
    db.add(db_chat)
    db.commit()
    db.refresh(db_chat)
    print(db_chat)
    
    # db.add(db_chat)
    # db.commit()
    # db.refresh(db_chat)

2025-06-08 11:13:38,072 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-06-08 11:13:38,074 INFO sqlalchemy.engine.Engine [raw sql] {}


2025-06-08 11:13:38,077 INFO sqlalchemy.engine.Engine select current_schema()
2025-06-08 11:13:38,078 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-08 11:13:38,080 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-06-08 11:13:38,081 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-08 11:13:38,082 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-08 11:13:38,090 INFO sqlalchemy.engine.Engine SELECT chat_sessions.id AS chat_sessions_id, chat_sessions.label AS chat_sessions_label, chat_sessions.messages AS chat_sessions_messages, chat_sessions.created_at AS chat_sessions_created_at, chat_sessions.updated_at AS chat_sessions_updated_at 
FROM chat_sessions 
WHERE chat_sessions.id = %(pk_1)s::UUID
2025-06-08 11:13:38,092 INFO sqlalchemy.engine.Engine [generated in 0.00224s] {'pk_1': UUID('da06a6dd-63df-410e-933c-e3c0e6c0ba5f')}
2025-06-08 11:13:38,101 INFO sqlalchemy.engine.Engine UPDATE chat_sessions SET messages=%(messages)s::JSONB, updated_at=%(updated_at)s W

In [19]:
print(db_chat.model_dump_json(indent=4))

{
    "updated_at": "2025-06-08T09:13:38.097353",
    "label": "Demo Contract Session",
    "created_at": "2025-06-07T21:50:07.117765",
    "id": "da06a6dd-63df-410e-933c-e3c0e6c0ba5f",
    "messages": [
        {
            "role": "user",
            "content": "Hello from client!",
            "timestamp": "2025-06-08T08:46:42.363283+00:00"
        },
        {
            "role": "assistant",
            "content": "Hello from assistant!",
            "timestamp": "2025-06-08T09:02:50.700660+00:00"
        },
        {
            "role": "user",
            "content": "Hello from client!",
            "timestamp": "2025-06-08T09:06:47.523262+00:00"
        },
        {
            "role": "assistant",
            "content": "Hello from ASSISTANT!",
            "timestamp": "2025-06-08T09:13:38.097114+00:00"
        }
    ]
}
